# Programming Assignment 1
## model transformers:
### 4. Semistructured to Property Graph
Each of the six groups should choose one and implement it in Python with a README file and some documentation. The implementation can be delivered in one of three ways: a command line utility, a Python package, or a REST API. A model transformer will take as input (a) a data file (e.g., CSV) of a specific model, and (b) a mapping configuration (in class we have seen several examples -- see the Model Transformation slides) and output a transformed data file. The developing group should also produce a sample input data file and its corresponding output data file. If any implementation uses an existing library or has some dependency (e.g., it uses Java), it should be mentioned in the README file so that the users can configure their environment accordingly. 

In [1]:
#pip install jsonpath-ng
import json
from jsonpath_ng.ext import parse

In [2]:
# Load people.json as Data Input-I
with open('./people.json') as people_json:
    people = json.load(people_json)

people

{'People': [{'id': 1,
   'Name': 'Max',
   'Age': 20,
   'Gender': 'Male',
   'friends': [2, 3]},
  {'id': 2, 'Name': 'Alice', 'Age': 25, 'Gender': 'Female', 'friends': [1]},
  {'id': 3, 'Name': 'Bob', 'Age': 22, 'Gender': 'Male', 'friends': [1, 4]},
  {'id': 4, 'Name': 'Joe', 'Age': 52, 'Gender': 'Male', 'friends': [3]}]}

In [3]:
# Exploration : Find People Gender where Name conatins Max 
jsonpath_expression = parse("$.People[?(@.Name[:] =~ 'Max')].Gender")

for match in jsonpath_expression.find(people):
    print(match.value)

Male


In [4]:
# Load people.json as Data Input-I
# Load the mapping configuration-mapping_config-4.json
# Mapping Logic – Nodes # Each row on our People becomes a node with People as the label.
# Mapping Logic – Edges # Join between People and itself (unary relationship) becomes a relationship named Friends (where People have friends ).
#with open('/home/prshukla/work/DSE203/mapping_config-4.json', 'r') as mapping_file:

# Source Data
data = people

# Load the mapping configuration-mapping_config-4.json
with open("mapping_config-4.json", "r") as mapping_file:
    mapping_config = json.load(mapping_file)

# Initialize the property graph JSON structure
property_graph = {"graph": {"nodes": [], "edges": []}}

# function to create a node in the property graph
def create_node(node_id, node_data):
    node = {"id": node_id, "label": [node_data["Name"]], "properties": {}}
    for key, value in node_data.items():
        if key != "id":
            property_name = mapping_config["People"].get(key, key)
            node["properties"][str(property_name)] = {"datatype": "string", "data value": value}
    property_graph["graph"]["nodes"].append({"node": node})

# function to create a relationship in the property graph
def create_relationship(source_id, target_id, relationship_type):
    edge = {"source": source_id, "target": target_id, "label": relationship_type, "properties": {}}
    property_graph["graph"]["edges"].append({"edge": edge})

# mapping to friend IDs and create nodes and relationships
for person in data["People"]:
    create_node(person["id"], person)
    for friend_id in person.get("friends", []):
        friend_id_mapped = mapping_config["People"]["friends"].get(str(friend_id), friend_id)
        create_relationship(person["id"], int(friend_id_mapped), "FRIENDS")

# Output the property graph JSON
with open("output_property_graph.json", "w") as output_file:
    json.dump(property_graph, output_file, indent=2)

property_graph


{'graph': {'nodes': [{'node': {'id': 1,
     'label': ['Max'],
     'properties': {'Name': {'datatype': 'string', 'data value': 'Max'},
      'Age': {'datatype': 'string', 'data value': 20},
      'Gender': {'datatype': 'string', 'data value': 'Male'},
      "{'_type': 'FRIENDS', '_source': 'id', '_target': 'friends'}": {'datatype': 'string',
       'data value': [2, 3]}}}},
   {'node': {'id': 2,
     'label': ['Alice'],
     'properties': {'Name': {'datatype': 'string', 'data value': 'Alice'},
      'Age': {'datatype': 'string', 'data value': 25},
      'Gender': {'datatype': 'string', 'data value': 'Female'},
      "{'_type': 'FRIENDS', '_source': 'id', '_target': 'friends'}": {'datatype': 'string',
       'data value': [1]}}}},
   {'node': {'id': 3,
     'label': ['Bob'],
     'properties': {'Name': {'datatype': 'string', 'data value': 'Bob'},
      'Age': {'datatype': 'string', 'data value': 22},
      'Gender': {'datatype': 'string', 'data value': 'Male'},
      "{'_type': 'FRIEND